In [11]:
import argparse, random, numpy as np, torch, gc
from torch import nn, optim
from torch.utils.data import DataLoader, random_split
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torchvision import transforms, models
from torchvision.datasets import EuroSAT
from tqdm import tqdm
from contextlib import nullcontext
import matplotlib.pyplot as plt

In [12]:
# ---------- 1. 參數 ----------
parser = argparse.ArgumentParser()
parser.add_argument('--data_dir', type=str, default='./data')
parser.add_argument('--batch_size', type=int, default=64)
parser.add_argument('--epochs', type=int, default=64)
parser.add_argument('--lr', type=float, default=5e-4)
parser.add_argument('--accum_steps', type=int, default=1,
                    help='梯度累積步數')
parser.add_argument('--variant', '-v', choices=['s', 'm', 'l'], default='m')
parser.add_argument('--freeze_backbone', action='store_true')
args, _ = parser.parse_known_args()

# ---------- 2. 環境 ----------
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
torch.manual_seed(42); random.seed(42); np.random.seed(42)

try:
    from torch.amp import autocast as _ac
    def amp_ctx(): return _ac(device_type=DEVICE) if DEVICE == 'cuda' else nullcontext()
except ImportError:
    from torch.cuda.amp import autocast as _ac
    def amp_ctx(): return _ac() if DEVICE == 'cuda' else nullcontext()

from torch.cuda.amp import GradScaler
scaler = GradScaler(enabled=(DEVICE == 'cuda'))

# ---------- 3. 資料 ----------
tfm = transforms.Compose([
    transforms.Resize(224, antialias=True),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])
dataset = EuroSAT(args.data_dir, download=True, transform=tfm)
n_total = len(dataset)
n_val = int(n_total * 0.15)  # 確保是整數
n_test = int(n_total * 0.15)  # 確保是整數
n_train = n_total - n_val - n_test

train_set, val_set, test_set = random_split(
    dataset, [n_train, n_val, n_test],
    generator=torch.Generator().manual_seed(42)
)


def make_loader(split, bs):
    return DataLoader(split, batch_size=bs,
                      shuffle=(split is train_set),
                      num_workers=2, pin_memory=True)

train_loader = make_loader(train_set, args.batch_size)
val_loader   = make_loader(val_set,   args.batch_size)
test_loader  = make_loader(test_set,  args.batch_size)
num_classes = len(dataset.classes)

<ipython-input-12-e8becc1ec16e>:25: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler(enabled=(DEVICE == 'cuda'))


In [13]:
# ---------- 4. 模型 ----------
v = args.variant
if v == 's':
    net = models.efficientnet_v2_s(
        weights=models.EfficientNet_V2_S_Weights.IMAGENET1K_V1)
elif v == 'm':
    net = models.efficientnet_v2_m(
        weights=models.EfficientNet_V2_M_Weights.IMAGENET1K_V1)
else:
    net = models.efficientnet_v2_l(
        weights=models.EfficientNet_V2_L_Weights.IMAGENET1K_V1)

net.classifier[1] = nn.Linear(net.classifier[1].in_features, num_classes)

if args.freeze_backbone:
    for n, p in net.named_parameters():
        if not n.startswith('classifier'):
            p.requires_grad = False
    print('Backbone frozen；僅微調分類頭')

net = net.to(DEVICE)
criterion = nn.CrossEntropyLoss()
opt = optim.AdamW([p for p in net.parameters() if p.requires_grad],
                  lr=args.lr, weight_decay=1e-4)
sched = ReduceLROnPlateau(opt, 'min', patience=2, factor=0.5, verbose=True)

Downloading: "https://download.pytorch.org/models/efficientnet_v2_m-dc08266a.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_v2_m-dc08266a.pth
100%|██████████| 208M/208M [00:01<00:00, 190MB/s]


In [ ]:
# ---------- 5. 訓練 ----------
history = {'train_loss': [], 'val_loss': [], 'train_acc': [], 'val_acc': []}

def forward_step(x, y):
    with amp_ctx():
        pred = net(x)
        loss = criterion(pred, y)
    return loss, pred

best_val = 0; stale = 0
for ep in range(1, args.epochs + 1):
    net.train()
    tloss = tcorrect = ttotal = 0
    opt.zero_grad(set_to_none=True)

    for idx, (x, y) in enumerate(tqdm(train_loader, desc=f'Epoch {ep}/{args.epochs}')):
        x, y = x.to(DEVICE, non_blocking=True), y.to(DEVICE, non_blocking=True)
        try:
            loss, pred = forward_step(x, y)
        except RuntimeError as e:
            if 'out of memory' in str(e):
                print('CUDA OOM，調小 --batch_size 或加大 --accum_steps')
                torch.cuda.empty_cache(); gc.collect()
                raise e
        scaler.scale(loss / args.accum_steps).backward()
        tcorrect += (pred.argmax(1) == y).sum().item()
        ttotal   += y.size(0)
        tloss    += loss.item() * y.size(0)

        if (idx + 1) % args.accum_steps == 0:
            scaler.step(opt); scaler.update()
            opt.zero_grad(set_to_none=True)

    train_loss = tloss / ttotal
    train_acc  = tcorrect / ttotal

    # ----- 驗證 -----
    net.eval()
    vloss = vcorrect = vtotal = 0
    with torch.no_grad():
        for x, y in val_loader:
            x, y = x.to(DEVICE, non_blocking=True), y.to(DEVICE, non_blocking=True)
            loss, pred = forward_step(x, y)
            vloss    += loss.item() * y.size(0)
            vcorrect += (pred.argmax(1) == y).sum().item()
            vtotal   += y.size(0)
    val_loss = vloss / vtotal
    val_acc  = vcorrect / vtotal

    # 紀錄
    history['train_loss'].append(train_loss)
    history['val_loss'].append(val_loss)
    history['train_acc'].append(train_acc)
    history['val_acc'].append(val_acc)

    sched.step(val_loss)

    if val_acc > best_val:
        best_val, stale = val_acc, 0
        torch.save(net.state_dict(), f'best_effv2_{v}.pt')
    else:
        stale += 1
        if stale >= 5:
            print('Early stopping'); break

    print(f'Epoch {ep:02d} | '
          f'train_loss {train_loss:.4f} train_acc {train_acc:.4f} | '
          f'val_loss {val_loss:.4f} val_acc {val_acc:.4f}')

Epoch 1/64:  21%|██        | 62/296 [00:35<02:08,  1.82it/s]

In [ ]:
# ---------- 6. 繪圖 ----------
epochs = range(1, len(history['train_loss']) + 1)

# Loss 曲線
plt.figure()
plt.plot(epochs, history['train_loss'], label='train_loss')
plt.plot(epochs, history['val_loss'],   label='val_loss')
plt.xlabel('Epoch'); plt.ylabel('Loss'); plt.title('Loss Curve'); plt.legend()
plt.savefig('loss_curve.png', dpi=300, bbox_inches='tight')   # ← 新增
plt.show()

# Accuracy 曲線
plt.figure()
plt.plot(epochs, history['train_acc'], label='train_acc')
plt.plot(epochs, history['val_acc'],   label='val_acc')
plt.xlabel('Epoch'); plt.ylabel('Accuracy'); plt.title('Accuracy Curve'); plt.legend()
plt.savefig('accuracy_curve.png', dpi=300, bbox_inches='tight')  # ← 新增
plt.show()


In [ ]:
# ---------- 7. 測試 ----------
from sklearn.metrics import classification_report
net.load_state_dict(torch.load(f'best_effv2_{v}.pt', map_location=DEVICE))
net.eval()
preds, labels = [], []
with torch.no_grad():
    for x, y in test_loader:
        x = x.to(DEVICE, non_blocking=True)
        _, pred = forward_step(x, y.to(DEVICE))
        preds.extend(pred.argmax(1).cpu().numpy())
        labels.extend(y.numpy())

print(classification_report(labels, preds, target_names=dataset.classes, digits=4))